In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger

from model import TPALSTM
from other_models import LSTM
import pandas as pd
from dataset import ElectricityDataModule

In [ ]:
data_df = np.load('data/kbest_dataset.npy')
data_df.shape

In [ ]:
data_splits = {
    "train": 0.7,
    "val": 0.15,
    "predict": 0.15
}

elec_dm = ElectricityDataModule(
    dataset_splits=data_splits,
    batch_size=256,
    window_size=24,
    pred_horizon=5,
    data_style="kbest"
)

In [ ]:
run_name = "5ts-kbest"

**LSTM**

In [ ]:
wandb_logger_lstm = WandbLogger(
    name=f'{run_name}-LSTM',
    save_dir='logs',
    project='Time-Series project',
    log_model=True
)

lstm_trainer = pl.Trainer(
    max_epochs=80,
    # accelerator='gpu',
    strategy='auto',
    devices=1,
    logger=wandb_logger_lstm
)

In [ ]:
lstm = LSTM(
    input_size=data_df.shape[1],
    lstm_hid_size=200,
    linear_hid_size=100,
    output_horizon=5,
    n_layers=1,
    lr=1e-3
)

In [ ]:
lstm_trainer.fit(lstm, elec_dm)
wandb_logger_lstm.experiment.finish()

**Stacked LSTM**

In [ ]:
wandb_logger_stacked_lstm = WandbLogger(
    name=f'{run_name}-Stacked LSTM',
    save_dir='logs',
    project='Time-Series project',
    log_model=True
)

stacked_lstm_trainer = pl.Trainer(
    max_epochs=80,
    accelerator='gpu',
    strategy='auto',
    devices=1,
    logger=wandb_logger_stacked_lstm
)

In [ ]:
stacked_lstm = LSTM(
    input_size=data_df.shape[1],
    lstm_hid_size=200,
    linear_hid_size=100,
    output_horizon=5,
    n_layers=1
)

In [ ]:
stacked_lstm_trainer.fit(stacked_lstm, elec_dm)
wandb_logger_stacked_lstm.experiment.finish()

**TPA-LSTM**

In [ ]:
wandb_logger_tpalstm = WandbLogger(
    name=f'{run_name}-TPA-LSTM-2-filters',
    save_dir='logs',
    project='Time-Series project',
    log_model=True
)

tpalstm_trainer = pl.Trainer(
    max_epochs=80,
    # accelerator='gpu',
    strategy='auto',
    devices=1,
    logger=wandb_logger_tpalstm
)

In [ ]:
tpa_lstm = TPALSTM(
    input_size=data_df.shape[1],
    output_horizon=1,
    hidden_size=50,
    output_horizon=5,
    obs_len=24,
    n_layers=2
)

In [ ]:
tpalstm_trainer.fit(tpa_lstm, elec_dm)
wandb_logger_tpalstm.experiment.finish()